# Determining the Effect of Earlier Weaning off Mechanical Ventilation
This project aims to use causal inference to estimate the effects of weaning patients off mechanical ventilation sooner than normal, following the approach described in [Kennedy, 2018](https://arxiv.org/pdf/1704.00211.pdf). In our dataset we consider patients who have been on mechanical ventilation for 24 hours (our start criteria).

In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ALL_WEANING_PREDICTORS is a list of all covariates intially identified as potentially useful in predicting probability of weaning
# group_by_stay is a helper function that groups a dataframe by the column 'stay_id'
from common import (
    ALL_WEANING_PREDICTORS,
    group_by_stay,
    remove_outliers,
    remove_extremes,
)

np.random.seed(0)

First, we exclude variables from consideration if they are missing more than 10% of the time in post-baseline times. Post-baseline in this case means we filter out records from patients at time points before our start criteria.

```
>>> full_df = pd.read_csv('full_weaning_dataset.csv')
>>> post_baseline_df = full_df.loc[full_df['hour_baseline'] >= 0]
```
We sample a subset of patiets from this post-baseline dataframe and use that as our dataset for this notebook.
```
>>> stay_ids = post_baseline_df['stay_id'].unique()
>>> print(len(stay_ids))
8055
```
Get a random sample of about 5% of the stay_ids.
```
>>> stays = np.random.choice(stay_ids, size=500)
>>> post_baseline_df.loc[post_baseline_df['stay_id'].isin(stays)].to_csv('sampled_post_baseline.csv')
```

In [2]:
# Load in the sampled dataset
df = pd.read_csv('sampled_post_baseline.csv')

Before testing which variables are missing, we fill missing urine output (`urine_output`) and vasopressor (`rate_std`) values with 0, since missing entries for these two columns means the value is 0 in this dataset. We do this to make sure we don't exclude the columns from the dataset when we shouldn't.

In [3]:
df.loc[:, ['urine_output', 'rate_std']] = df[['urine_output', 'rate_std']].fillna(0)

Then, we forward fill all columns except for `amount` on a per-patient basis. Their values may only be measured occasionally, which is fine since the previous measurement of these values is important, not whether they were measured recently.

In [4]:
for c in ALL_WEANING_PREDICTORS:
    if c == 'amount': continue
    df.loc[:, c] = group_by_stay(df, c).ffill()

In [5]:
LEN_DF = len(df)
cols_to_exclude = set()
for c in ALL_WEANING_PREDICTORS:
    na_proportion = df[c].isna().sum() / LEN_DF
    if na_proportion >= 0.1:
        cols_to_exclude.add(c)
        print(f"{c}: {round(na_proportion, 3)}")

fio2: 0.25
carboxyhemoglobin: 0.931
methemoglobin: 0.945
ALBUMIN: 0.24
BANDS: 0.655
BILIRUBIN: 0.147
marital_status: 0.119


Now that we've identified which columns should be excluded from our analysis, we can create a new list of predictors.

In [6]:
weaning_predictors = [c for c in ALL_WEANING_PREDICTORS if c not in cols_to_exclude]

The next step is to process the data and remove outliers. The `remove_outliers` function essentially writes over implausible values with NaN.

In [7]:
df.loc[:, 'amount'] = df['amount']/1000
df.loc[:, 'last_amount'] = df['last_amount']/1000

remove_outliers(df)

Now, we can standardize our predictors and create a model for the probability of weaning.

In [8]:
# If a column for the measurement of a given value at time t-1 exists, add it to the predictors
for c in weaning_predictors:
    try:
        df['last_' + c]
        weaning_predictors.append('last_' + c)
    except KeyError:
        pass

# Standardize predictors
numeric = df[weaning_predictors].select_dtypes(exclude='object').columns
df.loc[:, numeric] = StandardScaler().fit_transform(df[numeric])
# Sanity check to make sure the DataFrame was actually standardized
df[numeric].agg(['mean', 'std'])

,tidal_volume_set,tidal_volume_observed,plateau_pressure,ventilator_type,peep_set,tidal_volume_set,tidal_volume_observed,plateau_pressure,ventilator_type,peep_set,...,last_CREATININE,last_PLATELET,last_PTT,last_INR,last_PT,last_BUN,last_WBC,last_urine_output,last_GLUCOSE,last_weight
mean,1.654549e-16,-4.275184e-16,1.382913e-16,-6.298554e-17,1.073852e-16,1.654549e-16,-4.275184e-16,1.382913e-16,-6.298554e-17,1.073852e-16,...,-1.912502e-16,-2.390694e-17,3.776379e-16,6.553161e-16,2.016357e-16,-2.702597e-17,2.261550e-16,-1.631025e-17,1.101768e-16,-1.610924e-16
std,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,...,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000008e+00,1.000005e+00,1.000005e+00


Before modeling the probability of weaning, we'll impute medians and remove extreme values from the covariates.

In [9]:
for c in numeric:
    df.loc[:, c].fillna(df[c].median(), inplace=True)

remove_extremes(df)


In [10]:
MODEL_PARAMS = {
    'penalty': 'l2',
    'max_iter': 1000,
}
def _model(rows, predictors, label, options=MODEL_PARAMS):
    """Given the desired rows, predictors of a category, and label of that category, return a fitted sklearn logistic regressor."""
    # Encode any necessary columns that aren't encoded as integers
    data = df.loc[rows, predictors + [label]]
    for col in data.select_dtypes(include='object'):
        data.loc[:, col] = LabelEncoder().fit_transform(data[col])
    return LogisticRegression(**options).fit(data[predictors], data[label])